##Using Theano for Optimization

#What you'll learn
1.  Basic principals of python Theano package
2.  How to implement gradient descent using Theano

#Topics covered
1.  What is Theano and Why use it.
2.  Theano basics
    -Variable types and Data Structures
    -Debugging with Theano
    -Using Theano "function"
    -Symbolic differentiation
3.  Solving Optimization problems with Theano

#Preparation for class
1.  Install Theano 0.7.0 (latest Anaconda distro)
2.  Take a pass through tutorial at the following link:
http://nbviewer.ipython.org/github/craffel/theano-tutorial/blob/master/Theano%20Tutorial.ipynb
3.  Look over the definition and description of tensors on wiki page:
https://en.wikipedia.org/wiki/Tensor

#What is Theano and why use it? 

Theano is a software package developed at University of Montreal for solving machine learning
problems - neural nets in particular.  Theano provides enough structure to make programming a neural net relatively efficient in terms of lines of code required and readability of the resulting code.  It has flexibility to deal with a wide variety of different ML algorithm types and architectures.  For example, you'll see how easy it is to change from basic gradient descent to one of the accelerated first order methods that you've used.  Most remarkably, Theano lets you program and test your ML solution in a conventional single processor environment and then to alter the compute environment (e.g. to a graphic processor unit - GPU) in order to speed up training.  This allows you to solve very large problems within practical time constraints.  

Theano code looks like programming in python, but it has a somewhat different character.  The statements you write using Theano object are represented symbolically and not evaluated until required.  When evaluation is required, then the symbolic representation is compiled into C code for execution.  This results in code that's fast to execute and code that targets the compute environment that you use.  

##Basics
#Algebraic Data Structures (Algebraic Objects)
The tutorial referenced above (by craffel) comments that writing Theano code is more like writing algebraic equations than writing code.  For example, the equation $a = b + c$ makes sense for a lot of different types of objects.  If a, b, and c are all scalars (real numbers) then the equation makes sense.  It also makes sense if they are vectors of the same dimension or if they are matrices (or if they are real valued functions).  The meaning of the "+" is different for these different cases, but its meaning always stems from addition of two real numbers.  Vector addition is component by component addition of the real numbers making up the vectors and matrix addition is similarly component by component addition.  Here's how Theano defines these quantities. 

In [1]:
__author__ = 'mike.bowles'

import theano
from theano import tensor as T
from theano import function, shared
import math
import numpy as np

#Algebraic data structures (objects)
x = T.dscalar('x')
y = T.dscalar('y')
vec1 = T.vector('vec1')
vec2 = T.vector('vec2')
mat1 = T.matrix('mat1')
mat2 = T.matrix('mat2')

print type(x)
print vec1.type

<class 'theano.tensor.var.TensorVariable'>
TensorType(float64, vector)


Theano uses tensors as a class that encompasses a variety of familiar algebraic objects.  As indicated on https://en.wikipedia.org/wiki/Tensor, tensors encompass real numbers, vectors of real numbers, matrices of real numbers and 3-dimensional objects that can be visualized as a stack of matrices.  The number of dimensions of the object are called the order or degree of the tensor.  Thus a scalar (real number) is a 0th order tensor and an ordinary matrix of real numbers is a 2nd order tensor.  

#Q's

1.  Explain how a 2nd order tensor might be useful for holding a B&W image.  
2.  What could you use for holding a color image?

#Define algebraic relationships between algebraic objects
The code below shows some easy examples.  

In [2]:
#define some algebraic relationships
z1 = x + y
z2 = vec1 + vec2
z3 = mat1 + mat2

#What happens if you try to add different types of objects?
z4 = mat1 + x

print z1.type
print z2.type
print z3.type
print z4.type



TensorType(float64, scalar)
TensorType(float64, vector)
TensorType(float64, matrix)
TensorType(float64, matrix)


Were you expecting for Theano to barf on adding a scalar to a matrix?  What happens in this case is called "broadcasting".  That's numpy-speak for replicating or otherwise extending the smaller dimensioned object to match the dimension of the larger dimensioned object.  Sometimes it's handy to be able to add a constant to all the elements of a matrix.  

#Theano function
Theano.function plays a special role.  When you encapsulate theano symbolic manipulations in a function theano compiles the symbolic manipulations into C-code in order to be able to evaluate the function.  

In [3]:
addScalar = function([x, y], z1)
addScalar(2.0, 3.0)

array(5.0)

This version of function takes two arguments.  The first is a list containing the input variable names.  The second is a variable from the LHS of an equation relating the input variable to a result.  It may be helpful to recall that Theano is like writing mathematical equations.  The symbolic inputs and their relations to outputs are defined by the equations that preceed the function definition.  

#Q's

1.  Build a function addVector that adds two vectors and exercise it using a couple of np.arrays.  
2.  Build a function that adds a scalar to a matrix and define a matrix and a scalar to try it out.  What is the result?

Here are the results from applying various scalar functions to theano objects.  

In [5]:
#arithmetic functions
v = x**2
sq = function([x], v)
sq(5)
#sq(np.array([1, 2, 3]))

array(25.0)

#Q's

1.  Uncomment the last statement in the code snippet above.  What happens when you execute the code?  Why?
2.  Write a new function that uses "**2" to square a vector and evaluate it on an example.  
3.  What does "**2" do to a vector?  

Apparently **2 is an overloaded function in theano.  It has a definition for various theano objects.
4.  What happens if you use the math.sin() function to take the sine of a np.array()?  

Theano has a useful collection of transcendental functions that can be used generally on theano tensors.  Here are a few examples.

In [10]:
#try 
result = T.sin(0.5 * math.pi)
print result.eval()

result = T.sin(np.array([1.0, 2.0, 3.0]) * math.pi * 0.5)
print result.eval()


1.0
[  1.00000000e+00   1.22464680e-16  -1.00000000e+00]


#Q's

1.  So T.sin(np.array) yields an array.  What is the relationship between the elements of the input array and those of the output array?

In the basic tensor documentation 
http://deeplearning.net/software/theano/library/tensor/basic.html 
you'll find other mathematical functions.  Experiment with some of the functions under "Arithmetic" and "Mathematical".  

#Q's

1.  What does vArgMax = T.argmax(np.array([1, 2, 3, 4, 2])) do?  Construct a demonstration.
2.  Write an expression that returns the maximum element of a vector or matrix that you define explicitly.
3.  Write an expression that returns the element by element maximum of two vectors that you define explicitly.
4.  Use the pow() function to square a scalar.  Then use it to square a vector.  When you defined a function for squaring things we had to make a version for each theano tensor type (scalar, vector, matrix, etc.).  

You will make a lot of use of the T.dot() function.  This function performs dot product between two vectors and matrix product between two matrices.  Here are some examples showing how it works.  


In [12]:
#dot product of two vectors
result = T.dot(np.array([1, 2, 3]), np.array([2, 3, 4]))
print result.eval()

#two matrices that are not conformable 2x3 times 2x3 - uncomment to see result
#result = T.dot(np.array([[1, 2, 3], [4, 5, 6]]), np.array([[1, 2, 3], [4, 6, 5]]))
#print result.eval()

#transpose second matrix
result = T.dot(np.array([[1, 2, 3], [4, 5, 6]]), np.transpose(np.array([[1, 2, 3], [4, 6, 5]])))
print result.eval()
sin(x)

20
[[14 31]
 [32 76]]


#Q's

1.  Un-comment the two lines of code in the snippet above.  What error does that provoke?  Why?

#Shared Variables

Shared variables are a little different than the symbollic variables that you've seen so far.  Symbollic variables have several properties.  

1.  Their memory is shared across different functions.  This has several effects.  It means that they maintain state.  If one function changes a shared variable, then the new value is what will get used in subsequent functions.  You'll use shared variables for weights.  That enables you to iteratively take gradient descent steps to by making repeated calls to a training function.  

2.  Sharing memory also means that shared variables get allocated common memory when they are compiled for execution on GPU (for example).  All of the processors have access to the common memory.

3.  Shared variables cannot be used as inputs to a function.  Since they've got explicit values they don't need to be input to a function.  Any calculations involving them will use the explicit.  

4.  The values of a shared variable can be altered or extracted using set and get functions.

5.  Shared variables can be used in expressions like other symbolic variables.

Here's some simple examples using shared variables.  

In [25]:
shared_var = shared(0.0)
print shared_var.type()
print shared_var.get_value()
shared_var.set_value(2.0)
print shared_var.get_value()

<TensorType(float64, scalar)>
0.0
2.0


Shared variables are defined, they don't have to be listed as inputs to a function.  Here's an example from tutorial listed in the reading material for today.  

In [27]:
#square the shared variable
print shared_var.get_value()
squared = shared_var**2
square = function([], squared)
shared_var.set_value(square())
print shared_var.get_value()
square()


4.0
16.0


array(256.0)

#In-class coding example

Use shared variable and a function you define to raise a number to 128.  You can use **2, but you can't use higher powers than 2.  You can repetitively apply **2.  For example x**128 = (x**64)**2.


#Update parameter for Theano function()

Theano was created to generate gradient descent solutions for training machine learning models.  This form of training involves iteratively refining model parameters (or weights) to optimize performance.  The model parameters are going to be shared variables so that several functions can have access to them.  To accomplish training, you define an update for the model parameters.  That might be simply subtracting the gradient (times a step size) for plain gradient descent or it could me one of the more exotic update methods that you've become familiar with.  Theano functions have an input parameter where you define the update that you want to use.  There are several equivalent ways to define the update.  Suppose that the model parameters are called "w" and the update you want to implement is to add and increment called "inc".  Then the update parameter can be specified either as updates=[[w, w + inc]] or updates={w: w + inc}.  Here's a code example:

In [34]:
#update option in theano.function
#update can be used to alter the value of a shared variable.

w = theano.shared(np.array(0.0))
print "initial value of w", w.eval()
update = [[w, w + 100]]
zTemp = x * w
newW = function([x], zTemp, updates=update)
#each call to newW adds 100 to the scalar w
#Notice that the function evaluation occurs
#before the update
print "newW function value",  newW(1.0)
print "weight value", w.get_value()

#calling newW again yields a calculation with the
#w-value that resulted from the last update.

print "newW function value",  newW(1.0)
print "weight value", w.get_value()

initial value of w 0.0
newW function value 0.0
weight value 100.0
newW function value 100.0
weight value 200.0


Notice that the function zTemp gets evaluated before the update to w.  The change in w shows up in the second call to newW.  

#In class coding examples

1.  Change the definition of the update from the list form to the dictionary form and rerun. 
2.  Make the shared object that is being updated a 2x1 matrix.  Initialize it at [0.0, 0.0] and add a fixed amount to both elements of the vector.  


#Symbolic calculus with Theano - grad function

Now here's a truly remarkable feature of Theano.  Theano will symbolically determines the gradients you need to update model weights.  This is done with the grad function and is one of the major benifits of the symbolic approach that Theano takes.  Having symbolic representations of the chains of calculation enable the use of symbolic differentiation for common function and the use of the rules of calculus (e.g. chain rule for function composition) for constructing gradients of ML cost functions with respect to model parameters.  Here are some examples that show how the grad function works.  

In [6]:
#calculus - taking derivatives (calculate gradient)

v = x**2
diff = function([x], T.grad(v, [x]))
print diff(0.0), diff(1.0)

z1 = T.sin(x)
diff2 = function([x], T.grad(z1, [x]))
print diff2(0), diff2(math.pi/2.0)

z2 = T.dot(np.array([1.0, 2.0]), vec2)
gradient = function([vec2], T.grad(cost=z2, wrt=vec2))
print gradient(np.array([3.3, 4.4]))

[array(0.0)] [array(2.0)]
[array(1.0)] [array(6.123233995736766e-17)]
[ 1.  2.]


#In-class coding exercises

1.  Generate the code for the gradient of sine with a vector argument.  You can either create a function similar to diff2 above but with a vector argument or you can make the argument to T.sin(x) a shared vector variable and do .get_value() on the result of grad.  Evaluate the gradient at the point $[0.0, \frac{\pi}{4}, \frac{\pi}{2}, \frac{3\pi}{4}, \pi]$.  

2.  Generate the code to calculate the gradient with respect to x of the quadratic form $QF = \frac12x^TAx+b^Tx$.  In your code, make the square matrix A, the vector b, and x inputs to the function.  